The extraction of information such as persons name, organisation, adresses, events, year, keywords are necessary for anonimty check of the text, to classify or cluster text etc.

In [ ]:
#importing necessary modules
import spacy
import os
import glob
import pandas as pd

In [ ]:
#assigning the NLP model
#https://spacy.io/usage/models
#you need to download the spacy model before starting this step. 
#More info on the website of spacy to download spacy model. 
#You can also load your own custom made model if you want

nlp_model=spacy.load("en_core_web_sm")

In [ ]:
#creation of a dataframe to include the columns 
df_PA4= pd.DataFrame(columns = ['Text No.','Entities', 'Labels'])
#input files path where your pdf files are
files_path= '/User/Documents/pdf_files_folder'
#reading the path
read_files=glob.glob(os.path.join(files_path,"*.pdf")) 
#looping over the pdf files
for files in read_files:
    #if the filename is clear and distinct
    from pathlib import Path
    #Extracting the basename of the file
    base = os.path.basename(files)
    #Extract the filename from the basename of the file
    filename=Path(base).stem
    #if the filename is not the official name  of the application
    #filename="DMP_"+str(i)
    text=''
    #using pdfplumber for processing the pdf files
    with pdfplumber.open(files) as pdf:
        for pdf_page in pdf.pages:
            single_page_text=pdf_page.extract_text()
            text=text+ '\n' + single_page_text
    tx="  ".join(text.split('\n'))
    #Load SpaCy Model
    docs=nlp_model(tx)
    #initialize entities and labels variables
    entities = []
    labels = []
    for ent in docs.ents:
        entities.append(ent)
        labels.append(ent.label_)
    #collecting the entities and the corresponding labels of the file in the loop in a dataframe
    df = pd.DataFrame({'Text No.':filename,'Entities':entities,'Labels':labels})
    #assigning all the datatypes within df as string
    df = df.astype('str')
    #dropping the duplicates
    df.drop_duplicates(subset=['Text No.','Entities','Labels'], keep="first", inplace=True)
    #appending data of the file in the loop 
    df_PA4 = df_PA4.append(df, ignore_index=True)
df_PA4.head()

In [ ]:
#make a table format so that all the entities are in columns and text no are in rows
table = df_PA4.pivot_table(index=['Text No.'],
                             columns=['Labels'],
                             values=['Entities'],
                             aggfunc=lambda x: ', '.join(str(v) for v in x))
table.tail(2)

In [ ]:
#saving the table as an excel file
table.to_excel(input('Name of the file followed by .xlsx '))